<a href="https://colab.research.google.com/github/ravindrakush11/LangChain/blob/main/LC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq uvicorn lida kaleido python-multipart

In [ ]:
!pip install -qq langchain langchain-cli langsmith

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.9/493.9 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.9 MB/s eta 0:00:00


In [ ]:
!pip install -qq "langserve[all]"

In [ ]:
!pip install -qq cohere tiktoken openai

In [ ]:
OPENAI_API_KEY= " "

## LLM/ChatModel

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key = 'OPENAI_API_KEY')

In [ ]:
llm = OpenAI(openai_api_key = OPENAI_API_KEY)
chat_model = ChatOpenAI(openai_api_key = 'OPENAI_API_KEY')

In [ ]:
from langchain.schema import HumanMessage

text = "Provide a recipe of cake for 5 year old B'day baby"
messages = [HumanMessage(context = text)]

##Prompt Template

In [ ]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("Provide a recipe of cake for {year} year old B'day {person}")
p = prompt.format(year = "5",
                  person = "baby")
p

"Provide a recipe of cake for 5 year old B'day baby"

In [ ]:
# Passing a list of messages
from langchain.prompts.chat import ChatPromptTemplate
template = "What are the present that I can give to  my {relation} for a {day}"

human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

c = chat_prompt.format_messages(relation = 'girl friend', day = 'valentine day', text = 'I like comedy')
c

[SystemMessage(content='What are the present that I can give to  my girl friend for a valentine day'),
 HumanMessage(content='I like comedy')]

##Output Parsers

In [ ]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
  """Parse the output of an LLM call (generated message) to a comma separated list"""


  def parse(self, text:str):
    """Parse the output of an LLM call"""
    return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("subject, verb, object, predicate")

['subject', 'verb', 'object', 'predicate']

##Composing with LCEL

In [ ]:
from typing import List

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
  """Parse the output of an LLM call to a comma-separated list."""

  def parse(self, text:str):
    """Parse the output of an LLM call"""
    return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""


human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chain = chat_prompt | ChatOpenAI | CommaSeparatedListOutputParser()
chain.invoke({"text": "color"})

## LangServe

In [ ]:
!pip install -qq langserve

In [ ]:
from fastapi import FastAPI
from langserve import add_routes

#2. App definition
app = FastAPI(
    title = "LangChain Server",
    version = "1.0",
    description = "A simple api server using Langchain's Runnable interfaces",
)

#3. Adding chain route
add_route(
    app,
    category_chain,
    path = '/category_chain'
)

if __name__ == "__main__":
  import uvicorn
  uvicorn.run(app, host = 'localhost', port=8000)

NameError: ignored

##Client

In [ ]:
from langserve import RemoteRunnable
remote_chain = RemoteRunnable("http://localhost:8000/category_chain/")
remote_chain.invoke({"text": "colors"})